In [10]:
import fxcmpy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import datetime as dt
import time
%matplotlib inline
from matplotlib import style
style.use('ggplot')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
con=fxcmpy.fxcmpy(config_file = 'fxcm.cfg')
print(con.get_instruments_for_candles())

['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'AUS200', 'Bund', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'CHN50', 'Copper', 'ESP35', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NOK', 'EUR/NZD', 'EUR/SEK', 'EUR/TRY', 'EUR/USD', 'EUSTX50', 'FRA40', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'GER30', 'HKG33', 'JPN225', 'NAS100', 'NGAS', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'SOYF', 'SPX500', 'TRY/JPY', 'UK100', 'UKOil', 'US30', 'USD/CAD', 'USD/CHF', 'USD/CNH', 'USD/HKD', 'USD/JPY', 'USD/MXN', 'USD/NOK', 'USD/SEK', 'USD/TRY', 'USD/ZAR', 'USDOLLAR', 'USOil', 'XAG/USD', 'XAU/USD', 'ZAR/JPY', 'BTC/USD', 'US2000', 'WHEATF', 'CORNF', 'EMBasket', 'JPYBasket', 'BCH/USD', 'ETH/USD', 'LTC/USD', 'XRP/USD']


In [4]:
token = '1a5119225ed0a512e50f2c1df26952208792e06a'
symbol = 'EUR/USD'
timeframe = "m5"      # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
fast_sma_periods = 10
slow_sma_periods = 20
close_on_opposing_signal = True
amount = 10
stop = -10
limit = 30

In [16]:
# Global Variables
pricedata = None
numberofcandles = 100

In [17]:
def Prepare():
    global pricedata
    
    print("Requesting Initial Price Data...")
    pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
    print(pricedata.tail())
    print("Initial Price Data Received..")
# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
    while True:
        currenttime = dt.datetime.now()
        if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
            Update()
        elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
            Update()
            time.sleep(240)
        elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
            Update()
            time.sleep(840)
        elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
            Update()
            time.sleep(1740)
        elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
            Update()
            time.sleep(3540)
        time.sleep(1)

def GetLatestPriceData():
    global pricedata
    # Normal operation will update pricedata on first attempt
    new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
    if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
        pricedata= new_pricedata
        return True
        
    counter = 0
    # If data is not available on first attempt, try up to 3 times to update pricedata
    while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
        print("No updated prices found, trying again in 10 seconds...")
        counter+=1
        time.sleep(10)
        new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
    if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
        pricedata = new_pricedata
        return True
    else:
        return False

In [ ]:
def labelChange(df, colname):
    df['changeDetected'] = 0
    sign = 0
    df.reset_index(drop=True, inplace=True)
    for i, row in df.iterrows():
        sign_l = np.sign(row['slope'])
#         print("Previous sign: {}  and current sign: {}".format(sign, sign_l))
        if sign_l>=0 and sign<=0:
            sign = sign_l
            df['changeDetected'].iloc[i:i+1,] = 1
        elif sign_l<=0 and sign>=0:
            sign = sign_l
            df['changeDetected'].iloc[i:i+1,] = -1
        else:
            df['changeDetected'].iloc[i:i+1,] = 0
    return df

def getSlope(x):
    n = len(x)
    y = np.arange(1, n+1, 1)
    slope,_ = np.polyfit(x, y, deg=1)
    return slope

In [ ]:
def Update():
    print(str(dt.datetime.now()) + "    " + timeframe + " Bar Closed - Running Update Function...")

#     # Calculate Indicators
#     iFastSMA = sma(pricedata['bidclose'], fast_sma_periods)
#     iSlowSMA = sma(pricedata['bidclose'], slow_sma_periods)
    
    data = pd.DataFrame(pricedata[['askclose', 'bidclose']].mean(axis=1), columns=['mid_close'])
    data['mid_close_mva4'] = data['mid_close'].rolling(4).mean()
    data['mid_close_mva5'] = data['mid_close'].rolling(5).mean()
    data['slope'] = data.mid_close_mva4.rolling(10).apply(getSlope)
    data = labelChange(df=data, colname='slope')
#     # Print Price/Indicators
#     print("Close Price: " + str(pricedata['bidclose'][len(pricedata)-1]))
#     print("Fast SMA: " + str(iFastSMA[len(iFastSMA)-1]))
#     print("Slow SMA: " + str(iSlowSMA[len(iSlowSMA)-1]))
    
    # TRADING LOGIC data['mid_close'][9:].values[0]
    # If Fast SMA crosses over Slow SMA, Open Buy Trade
    if data['changeDetected'][99:].values[0]==1:
        print("  BUY SIGNAL!")
        if close_on_opposing_signal and countOpenTrades("S") > 0:
            print(" Closing Sell Trade(s)...")
            exit("S")
        print("  Opening Buy Trade...")
        enter("B")
    # If Fast SMA crosses under Slow SMA, Open Sell Trade
    if data['changeDetected'][99:].values[0]==-1:
        print("  SELL SIGNAL!")
        if close_on_opposing_signal and countOpenTrades("B") > 0:
            print("  Closing Buy Trade(s)...")
            exit("B")
        print("  Opening Sell Trade...")
        enter("S")
        
    print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
    direction = True;
    if BuySell == "S":
        direction = False;
    try:
        opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount,time_in_force='GTC', order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop)
    except Exception as e:
        print("  Error Opening Trade.")
        print(e)
    else:
        print("  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
    openpositions = con.get_open_positions(kind='list')
    isbuy = True
    if BuySell == "S":
        isbuy = False
    for position in openpositions:
        if position['currency'] == symbol:
            if BuySell is None or position['isBuy'] == isbuy:
                print("  Closing tradeID: " + position['tradeId'])
                try:
                    closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
                except Exception as e:
                    print("  Error Closing Trade.")
                    print(e)
                else:
                    print("  Trade Closed Successfully.")
                    
def countOpenTrades(BuySell=None):
    openpositions = con.get_open_positions(kind='list')
    isbuy = True
    counter = 0
    if BuySell == "S":
        isbuy = False
    for position in openpositions:
        if position['currency'] == symbol:
            if BuySell is None or position['isBuy'] == isbuy:
                counter+=1
    return counter

In [ ]:
Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy

Requesting Initial Price Data...
                     bidopen  bidclose  bidhigh   bidlow  askopen  askclose  \
date                                                                          
2019-06-11 13:50:00  1.13163   1.13149  1.13171  1.13144  1.13173   1.13161   
2019-06-11 13:55:00  1.13149   1.13145  1.13156  1.13135  1.13161   1.13157   
2019-06-11 14:00:00  1.13144   1.13204  1.13217  1.13136  1.13156   1.13217   
2019-06-11 14:05:00  1.13205   1.13183  1.13244  1.13182  1.13218   1.13196   
2019-06-11 14:10:00  1.13183   1.13172  1.13193  1.13161  1.13196   1.13184   

                     askhigh   asklow  tickqty  
date                                            
2019-06-11 13:50:00  1.13182  1.13156     1372  
2019-06-11 13:55:00  1.13167  1.13147      985  
2019-06-11 14:00:00  1.13228  1.13147     1209  
2019-06-11 14:05:00  1.13252  1.13194     1505  
2019-06-11 14:10:00  1.13205  1.13173      970  
Initial Price Data Received..
2019-06-11 18:20:01.718688    m5 Bar Clo

C:\Users\Jameel\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Jameel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sign
  
C:\Users\Jameel\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Previous sign: 0  and current sign: nan
Previous sign: 0  and current sign: nan
Previous sign: 0  and current sign: nan
Previous sign: 0  and current sign: nan
Previous sign: 0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: 1.0
Previous sign: 1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and current sign: -1.0
Previous sign: -1.0  and cu